<a href="https://www.kaggle.com/code/pmtphamtuan/khaiphadl?scriptVersionId=254218333" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [10]:
paths = ['/kaggle/input/meteonet/NW_Ground_Stations/NW_Ground_Stations/NW_Ground_Stations_2016.csv',
         '/kaggle/input/meteonet/NW_Ground_Stations/NW_Ground_Stations/NW_Ground_Stations_2017.csv',
         '/kaggle/input/meteonet/NW_Ground_Stations/NW_Ground_Stations/NW_Ground_Stations_2018.csv']

In [11]:
num_cols = ['height_sta','dd', 'ff', 'precip','hu', 'td', 't', 'psl']
dtype = dict([(k,'float32') for k in num_cols])

In [12]:
def open_csv(path:str):
    df =  pd.read_csv(
      path,
      header = 0,
      dtype = dtype
  )
    return df

In [13]:
weather_data = pd.concat((open_csv(_) for _ in paths))

In [23]:
weather_data.to_csv('weather_data.csv', index=False)

In [8]:
weather_data.head()

NameError: name 'weather_data' is not defined

In [ ]:
weather_data = weather_data.loc[weather_data['lat']>48.4]
weather_data = weather_data.loc[weather_data['lon']>-1.6]

In [ ]:
weather_data['date'] = pd.to_datetime(weather_data['date'])

The weather dataset
The dataset contains 3 years of weather data collected from two main sources : ground stations and weather satellites. We'll build climate models which learn weather patterns from past observations and output one/multi timestep(s) forecasts. To limit latency we'll reduce the data space to approximately a quarter of the +60M observations reported in the dataset.These are weather parameters recorded by dozens of weather stations every 6 minutes from 2016 to 2018. We'll simplify the problem by training our models at the station level.

Input features

number_sta ----------> ground station ID

lat : latitude ----------> decimal degrees (10-1 °)

lon : longitude ----------> decimal degrees (10-1 °)

height_sta ----------> station height meters (m)

date ----------> a datetime object ('YYYY-MM-DD HH: mm :ss')

dd ----------> Wind direction degrees (°)

ff ----------> Wind speed m.s-1

precip ----------> Precipitation during the reporting period kg.m2

hu ----------> Humidity percentage (%)

td ----------> Dew point Kelvin (K)

t ----------> Temperature Kelvin (K)

psl ----------> Pressure reduced to sea level Pascal (Pa)

In [ ]:
weather_data.info()

In [ ]:
weather_data.describe().round()

In [ ]:
weather_data['date'] = pd.to_datetime(weather_data['date'])

In [ ]:
values = {_:np.mean(weather_data[_]) for _ in num_cols}
weather_data = weather_data.fillna(value = values)

In [ ]:
weather_data.info()

In [ ]:
weather_data.head()

In [ ]:
loc = weather_data['number_sta'].sample(1).values[0]

In [ ]:
station_id = np.unique(weather_data['number_sta'])
coordinates = [
    [
        np.mean(weather_data.loc[weather_data['number_sta'] == k,'lat']),
        np.mean(weather_data.loc[weather_data['number_sta'] == k,'lon'])
    ]
                for k in station_id
]
stations = {k:v for k,v in zip(station_id,coordinates)}

In [ ]:
annual_rainfall = weather_data.groupby([weather_data['date'].dt.year,'number_sta'])['precip'].sum()
annual_rainfall = annual_rainfall.reset_index(1).groupby('number_sta')['precip'].mean()

In [ ]:
df = pd.DataFrame(annual_rainfall)
df['lat'] = [stations[_][0] for _ in df.index]
df['lon'] = [stations[_][1] for _ in df.index]
fig = px.scatter_mapbox(
    df, lat='lat', lon='lon',
    zoom = 6,
    color = 'precip',
    color_continuous_scale=px.colors.sequential.YlGnBu
)

print('Average annual precipitation in mm')
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(marker_size=12)
fig.show()

In [ ]:
df = df.reset_index(0)
df['height_sta'] = [weather_data.loc[weather_data['number_sta'] == _,'height_sta'].values[0] for _ in df['number_sta']]
df['Atm. pressure'] = [weather_data.loc[weather_data['number_sta'] == _,'psl'].values[0] for _ in df['number_sta']]
df['ff'] = weather_data.groupby([weather_data['date'].dt.year,'number_sta'])['ff'].mean().groupby('number_sta').mean().values
features = ['height_sta','Atm. pressure', 'ff']

In [ ]:
titles = ["Height of Station", "Pressure Reduced to Sea Level", "Wind Speed"]

fig, axs = plt.subplots(2, 2, sharey=True, figsize=[8, 8])

# Lặp qua các trục và các đặc trưng
for ax, i, title in zip(axs.flatten(), features, titles):
    sns.scatterplot(data=df, x=i, y='precip', ax=ax)
    ax.set_title(title)  # Đặt tiêu đề cho từng biểu đồ

# Loại bỏ ô trống thừa (nếu có)
plt.delaxes(axs[1, 1])

plt.tight_layout()
plt.show()

In [ ]:
def color_strong_corr(val):
    color = 'red' if (abs(val) > 0.1) & (abs(val) <1.0) else 'black'
    return 'color: %s' % color
df[['precip','height_sta','Atm. pressure','ff']].corr().\
    style.applymap(color_strong_corr)



In [ ]:
features = ['height_sta', 'Atm. pressure', 'ff', 'precip']

# Select only the columns of interest
df_features = df[features]

# Calculate the correlation matrix
corr_matrix = df_features.corr()

# Plot the correlation matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1, linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df = weather_data.set_index('date')
df['t'] = df['t'] - 273.5

In [ ]:
df

In [ ]:
fig = px.line(df['2016-01-01':'2018-12-30'][['t','hu']].resample('D').mean(),
             title = 'Temperature (°C) - Humidity ratio')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
fig = px.line(df['2016-01-01':'2018-12-30']['precip'].resample('7D').sum(),
             title = 'Overall weekly precipitation')
fig.update_xaxes(
    rangeslider_visible=True
)
fig.show()


In [ ]:
jan_2018 = df['2018-01-03':'2018-01-03'].reset_index()
jan_2018['ff'] = jan_2018['ff'] * 3.6
jan_2017 = df['2017-01-03':'2017-01-03'].reset_index()
jan_2016 = df['2016-01-03':'2016-01-03'].reset_index()

In [ ]:
fig,axs = plt.subplots(1,2, figsize = (12,6))
jan_2018.groupby(jan_2018['date'].dt.hour)["ff"].max().plot(kind = 'bar', rot=0,
                                                 title = "Max windspeed recorded 2018/01/03",
                                                 xlabel = 'Hour of the day',
                                                 ylabel = 'Windspeed in km/h',
                                                 ax = axs[0])
jan_2018.groupby(jan_2018['date'].dt.hour)["precip"].mean().plot(kind = 'bar', rot=0,
                                                 title = "Avg Rainfall per station 2018/01/03",
                                                 xlabel = 'Hour of the day',
                                                 ylabel = 'Precipitation in mm',
                                                 ax = axs[1])

In [ ]:
df = [jan_2018,jan_2017,jan_2016]
title = ["2018-01-03","2017-01-03","2016-01-03"]
fig, axs = plt.subplots(2,2, sharey = True, figsize = [10,10])
fig.suptitle("Cumulative hourly precipitation")
for ax,i,df,title in zip(axs.flatten(),features,df,title) :
    df.groupby(df['date'].dt.hour)["precip"].sum().cumsum().plot(kind = 'line',
                             title = title,
                             ylabel = 'Rainfall in mm',
                             xlabel = 'Hours of the day',
                             xticks = np.arange(0,26,2),
                             ax =  ax
                        )
plt.delaxes(axs[1,1])


In [ ]:
date_range = pd.date_range(start= '2016-01-01', end = '2018-12-31',
                          freq='D').strftime("%Y-%m-%d")

In [ ]:
import time

In [ ]:
init = time.time()

In [ ]:
def daily_forecast(loc) :
    df = weather_data.loc[weather_data['number_sta'] == loc]
    df = df.set_index('date')
    df['hours'] = [_.hour for _ in df.index]
    df['days'] = [_.dayofyear for _ in df.index]
    df['years'] = [_.year for _ in df.index]
    
   
    df['3'] = df['hours']%3
    df = df.loc[df['3'] == 0.0]
    df = pd.concat([
        df[_:_].drop_duplicates(subset = 'hours').reset_index(drop = True) for
        _ in date_range
                   ]
    )
    df = df[["height_sta","dd","ff","precip","hu","td","t","hours","days","years"]]
    
    return df

In [ ]:
features = ["height_sta","dd","ff","precip","hu","td","t","days"]
days0 = np.arange(1,365,2)
days1 = np.arange(2,365,2)
len(days0) == len(days1)


In [ ]:
def _3h_windowing(df:pd.DataFrame) :
    s_scaler = MinMaxScaler()
    
    days0 = np.arange(1,365,2)
    days1 = np.arange(2,365,2)
    X,y = [],[]
    
    for d0,d1 in zip(days0,days1) :
        x = df.loc[df['days'] == d0]
        z = df.loc[df['days'] == d1]
        
        if len(x) == len(z) :
            X.append(x)
            y.append(z)
            
    X = pd.concat(X)[features]
    y =  pd.concat(y)[features]
    
    X = pd.DataFrame(s_scaler.fit_transform(X))
    y = pd.DataFrame(s_scaler.fit_transform(y))
        
    return X,y, s_scaler

In [ ]:
def split_dataset(X:pd.DataFrame, y:pd.DataFrame) :
    
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle = True, random_state=42)
    
    X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, shuffle = True, random_state=7)
    
    return {
        'train_set': tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64).prefetch(2),
        'val_set' : tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64).prefetch(2),
        'test_set' : [X_test, y_test]
           }

In [ ]:
def model_history(model:'o', train_metrics:str, val_metrics:str, loss:str):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel(loss)
    plt.plot(model.epoch, np.array(model.history[train_metrics]),
           label='Train')
    plt.plot(model.epoch, np.array(model.history[val_metrics]),
           label = 'Val')
    plt.legend()

In [ ]:
df = daily_forecast(loc)

In [ ]:
df.head(5)

In [ ]:
df

**> Pyspark******

In [ ]:
!pip install pyspark

In [ ]:
df

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("WeatherForecasting").getOrCreate()

spark_df = spark.createDataFrame(df)

In [ ]:
spark_df

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline


feature_cols = ["height_sta", "dd", "ff", "precip", "hu", "td", "hours", "days", "years"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")


pipeline = Pipeline(stages=[assembler])


df_transformed = pipeline.fit(spark_df).transform(spark_df)


df_transformed.select("features").show(truncate=False)



In [ ]:
df_transformed

In [ ]:
# Show transformed data
df_transformed.select("features").show()

In [ ]:
# Split the data into train, validation, and test sets
train_set,test_set = df_transformed.randomSplit([0.8, 0.2], seed=1234)


In [ ]:
train_set

In [ ]:
test_set

In [ ]:
from pyspark.ml.regression import LinearRegression

# Initialize the model
lr = LinearRegression(featuresCol="features", labelCol="t")

# Train the model
lr_model = lr.fit(train_set)



In [ ]:
print(f"Number of iterations: {lr_model.summary.totalIterations}")


In [ ]:
# Make predictions on the validation set
test_predictions = lr_model.transform(test_set)



In [ ]:
test_predictions

In [ ]:
test_predictions.select("prediction", "t", "features").show()


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model on validation set
evaluator = RegressionEvaluator(labelCol="t", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(test_predictions)
print(f"test_predictions RMSE: {rmse}")

# Evaluate the model on test set
test_predictions = lr_model.transform(test_set)
test_rmse = evaluator.evaluate(test_predictions)
print(f"Test RMSE: {test_rmse}")


In [ ]:
test_predictions

In [ ]:
test_predictions.toPandas()

In [ ]:
test_predictions

In [ ]:
def plot(test_predictions_pd,titel="Test Set: Actual vs Predicted Linear Regression"): 
    plt.figure(figsize=(12, 6))
    plt.plot(test_predictions_pd["t"], label="Actual")
    plt.plot(test_predictions_pd["prediction"], label="Predicted")
    plt.plot(test_predictions_pd["t"] - test_predictions_pd["prediction"], 
             label="Residual (Actual - Predicted)", linestyle="--")
    plt.title(titel)
    plt.xlabel("Index")
    plt.ylabel("Temperature")
    plt.legend()
    plt.show()

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor


dt = DecisionTreeRegressor(featuresCol="features", labelCol="t")


dt_model = dt.fit(train_set)


rf = RandomForestRegressor(featuresCol="features", labelCol="t")


rf_model = rf.fit(train_set)


dt_predictions = dt_model.transform(test_set)
rf_predictions = rf_model.transform(test_set)


from pyspark.ml.evaluation import RegressionEvaluator


evaluator = RegressionEvaluator(labelCol="t", predictionCol="prediction", metricName="rmse")




In [ ]:
rf_feature_importances = pd.DataFrame({
'feature': feature_cols,
'importance': rf_model.featureImportances.toArray()
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(rf_feature_importances['feature'], rf_feature_importances['importance'])
plt.title('Mức độ quan trọng của các thuộc tính')
plt.xlabel('Tầm quan trọng')
plt.gca().invert_yaxis() # Hiển thị thuộc tính quan trọng nhất ở trên cùng
plt.show()

In [ ]:
dt_predictions.select("t", "prediction").show()

In [ ]:

dt_rmse = evaluator.evaluate(dt_predictions)
print(f"Decision Tree RMSE: {dt_rmse}")


rf_rmse = evaluator.evaluate(rf_predictions)
print(f"Random Forest RMSE: {rf_rmse}")

In [ ]:
dt_predictions

In [ ]:

dt_predictions_pd = dt_predictions.select("t", "prediction").toPandas()
rf_predictions_pd = rf_predictions.select("t", "prediction").toPandas()

In [ ]:
ln_predictions_pd=test_predictions.select("t", "prediction").toPandas()

In [ ]:
dt_predictions_pd

In [ ]:

dt_predictions_pd = dt_predictions.select("t", "prediction").toPandas()
rf_predictions_pd = rf_predictions.select("t", "prediction").toPandas()

dt_predictions_pd["residual"] = dt_predictions_pd["t"] - dt_predictions_pd["prediction"]
rf_predictions_pd["residual"] = rf_predictions_pd["t"] - rf_predictions_pd["prediction"]


plt.figure(figsize=(12, 6))


plt.subplot(1, 2, 1)
plt.scatter(dt_predictions_pd.index, dt_predictions_pd["residual"], color='orange', alpha=0.5)
plt.axhline(0, color='black', linewidth=1)
plt.title("Residuals for Decision Tree")
plt.xlabel("Index")
plt.ylabel("Residual (Actual - Predicted)")


plt.subplot(1, 2, 2)
plt.scatter(rf_predictions_pd.index, rf_predictions_pd["residual"], color='blue', alpha=0.5)
plt.axhline(0, color='black', linewidth=1)
plt.title("Residuals for Random Forest")
plt.xlabel("Index")
plt.ylabel("Residual (Actual - Predicted)")

plt.tight_layout()
plt.show()

In [ ]:
test_predictions_pd= test_predictions.select("t", "prediction").toPandas()
test_predictions_pd["residual"] = test_predictions_pd["t"] - test_predictions_pd["prediction"]

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 2)
plt.scatter(test_predictions_pd.index, test_predictions_pd["residual"], color='red', alpha=0.5)
plt.axhline(0, color='black', linewidth=1)
plt.title("Residuals for LinearRegression")
plt.xlabel("Index")
plt.ylabel("Residual (Actual - Predicted)")
plt.show()

In [ ]:
mae_evaluator = RegressionEvaluator(labelCol="t", predictionCol="prediction", metricName="mae")
dt_mae = mae_evaluator.evaluate(dt_predictions)
rf_mae = mae_evaluator.evaluate(rf_predictions)


In [ ]:
r2_evaluator = RegressionEvaluator(labelCol="t", predictionCol="prediction", metricName="r2")
dt_r2 = r2_evaluator.evaluate(dt_predictions)
rf_r2 = r2_evaluator.evaluate(rf_predictions)


In [ ]:
print("Decision Tree Performance:")
print(f"RMSE: {evaluator.evaluate(dt_predictions)}")
print(f"MAE: {mae_evaluator.evaluate(dt_predictions)}")
print(f"R2: {r2_evaluator.evaluate(dt_predictions)}")


print("\nRandom Forest Performance:")
print(f"RMSE: {evaluator.evaluate(rf_predictions)}")
print(f"MAE: {mae_evaluator.evaluate(rf_predictions)}")
print(f"R2: {r2_evaluator.evaluate(rf_predictions)}")



In [ ]:
model_fit.write().overwrite().save("models/Decision_Tree_model")
model_fit.write().overwrite().save("models/random_forest_model")


In [ ]:
r2_evaluator = RegressionEvaluator(labelCol="t", predictionCol="prediction", metricName="r2")
# dt_r2 = r2_evaluator.evaluate(test_predictions)
print("\nLinear Regression Performance:")
print(f"RMSE: {evaluator.evaluate(test_predictions)}")
print(f"MAE: {mae_evaluator.evaluate(test_predictions)}")
print(f"R2: {r2_evaluator.evaluate(test_predictions)}")

In [ ]:


model_fit.write().overwrite().save("models/Linear_Regression_model")